# GRG Supply Chain optimization - Linear Programming

This Model helps to decide what raw material to order from Current GRG supplier,

where to manufacture the products, 

how to deliver the manufactured products to the customers so that the overall cost is minimized.

# required data

Supplier stock : A table indicating how many units of each raw material each of the suppliers has in stock. -

Raw material costs: A table indicating how much each of the suppliers is charging per unit for each of the raw materials.

Raw material shipping A table indicating the shipping costs per unit of raw material (the units for each material are the same) from each supplier to each factory.

Product requirements : A table indicating the amount of raw material required to manufacture one unit of each of the products.

Production capacity A table indicating how many units of each product each of the factories is able to manufacture.

Production cost A table indicating the cost of manufacturing a unit of each product in each of the factories.

Customer demand A table indicating the number of units of each product that have been ordered by the customers

Shipping costs A table indicating the shipping costs per unit for delivering a product to the customer

In [3]:
from ortools.linear_solver import pywraplp
import pandas as pd

In [4]:
# 1. Loading the input data
sup_stock = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Supplier stock", index_col=0)
raw_material_cost = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Raw material costs", index_col=0)
raw_material_shipping = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Raw material shipping", index_col=0)
production_req = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Product requirements", index_col=0)
production_capacity = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Production capacity", index_col=0)
customer_demand = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Customer demand", index_col=0)
production_cost = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Production cost", index_col=0)
shipping_costs = pd.read_excel("C:/Users/JanithaPrathapa/Desktop/GRG- LINEAR_PROGRAMMING/data.xlsx", sheet_name = "Shipping costs", index_col=0)

In [5]:
# Changing NaN values to 0 for easy computation
customer_demand = customer_demand.fillna(0)
production_req = production_req.fillna(0)
sup_stock = sup_stock.fillna(0)
production_capacity = production_capacity.fillna(0)
raw_material_cost = raw_material_cost.fillna(0)
production_cost = production_cost.fillna(0)

In [6]:
# Getting list of factories 

factories = list(raw_material_shipping.columns )
print("Factories:\n",factories)
# Getting list of materials
materials = list(raw_material_cost.columns)
print("Materials: \n",materials)
# Getting list of suppliers
suppliers = list(raw_material_cost.index)
print("Suppliers: \n",suppliers)
#Getting list of products
products = list(production_req.index)
print("Products: \n",products)
#Getting list of customers
customers = list(customer_demand.columns)
print("Customers: \n",customers)

Factories:
 ['Factory A', 'Factory B', 'Factory C']
Materials: 
 ['Material A', 'Material B', 'Material C', 'Material D']
Suppliers: 
 ['Supplier A', 'Supplier B', 'Supplier C', 'Supplier D', 'Supplier E']
Products: 
 ['Product A', 'Product B', 'Product C', 'Product D']
Customers: 
 ['Customer A', 'Customer B', 'Customer C', 'Customer D']


In [7]:
# 2. Creating Decision Variables using OR Tools wrapper of the GLOP_LINEAR_PROGRAMMING solver.
solver = pywraplp.Solver('LPWrapper', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
orders = {}
for factory in factories:
    for material in materials:
        for supplier in suppliers:
            orders[(factory, material, supplier)] = solver.NumVar(0, solver.infinity(), 
                                          factory+"_"+material+"_"+supplier)
            
production_volume = {}
for factory in factories:
    for product in products: 
        production_volume[(factory, product)] = solver.NumVar(0, solver.infinity()  , factory+"_"+product)
        
delivery = {}
for factory in factories: 
    for customer in customers:
        for product in products: 
            delivery[(factory, customer, product)] = solver.NumVar(0, solver.infinity(), factory+"_"+customer+"_"+product)



In [8]:
# C. Define and implement the constraints that ensure factories produce more 
#    than they ship to the customers

for product in products: 
    for factory in factories:
        c = solver.Constraint(0, solver.infinity())
        c.SetCoefficient(production_volume[(factory, product)] , 1)
        for customer in customers:             
            c.SetCoefficient(delivery[(factory, customer, product)], -1)


In [9]:
## D. Define and implement the constraints that ensure that customer demand is met

for customer in customers: 
    for product in products:
        
        c = solver.Constraint(int(customer_demand.loc[product][customer]),int(customer_demand.loc[product][customer]))
        for factory in factories: 
            c.SetCoefficient(delivery[(factory,customer,product)], 1)

In [21]:
## E. Define and implement the constraints that ensure that suppliers have all ordered items in stock     


for supplier in suppliers: 
    for material in materials: 
        c = solver.Constraint(0, int(sup_stock.loc[supplier][material]))
        for factory in factories: 
            c.SetCoefficient(orders[(factory, material, supplier)],1)

In [11]:
# F. Define and implement the constraints that ensure that factories order 
#    enough material to be able to manufacture all items


for factory in factories:
    for material in materials:
        c = solver.Constraint(0,solver.infinity())
        for supplier in suppliers:
            c.SetCoefficient(orders[(factory, material, supplier)],1)
            for product in products:
                c.SetCoefficient(production_volume[(factory, product)], - production_req.loc[product][material])

In [12]:
### G.Define and implement the constraints that ensure that the manufacturing capacities are not exceeded
##
                                
for factory in factories: 
    for product in products: 
        c = solver.Constraint(0, int(production_capacity.loc[product][factory]))  
        c.SetCoefficient(production_volume[(factory, product)],1)

In [13]:
###   Define and implement the objective function.
cost = solver.Objective()
# Material Costs  + shipping costs 
for factory in factories: 
    for supplier in suppliers:
        for material in materials:
            cost.SetCoefficient(orders[(factory, material, supplier)] , 
                                       raw_material_cost.loc[supplier][material] + raw_material_shipping.loc[supplier][factory])
            

In [14]:
# production cost of each factory 
for factory in factories: 
    for product in products: 
        cost.SetCoefficient(production_volume[(factory, product)], int(production_cost.loc[product][factory]))

In [15]:
# shipping cost to customers 
for factory in factories: 
    for customer in customers:
        for product in products: 
            cost.SetCoefficient(delivery[(factory, customer, product)], int(shipping_costs.loc[factory][customer]))

In [16]:
# I. Solve the linear program and determine the optimal overall cost
            
cost.SetMinimization()
status = solver.Solve()

if status == solver.OPTIMAL:
    print("Optimal Solution Found")
print("Optimal Overall Cost: ", solver.Objective().Value())

# J. and K  Supplier Bill
print("\nSupplier Bill and order quantity")
print("****************************")
for factory in factories:
    print(factory,":")
    
    for supplier in suppliers:
        factory_cost = 0
        print("  ",supplier,":")
        for material in materials:
            print("\t",material,":", orders[(factory, material, supplier)].solution_value())
            
            factory_cost += orders[(factory, material, supplier)].solution_value() * raw_material_cost.loc[supplier][material]
            factory_cost += orders[(factory, material, supplier)].solution_value() * float(raw_material_shipping.loc[supplier][factory])
        print("  ",supplier," Bill: ", factory_cost)

Optimal Solution Found
Optimal Overall Cost:  49315.0

Supplier Bill and order quantity
****************************
Factory A :
   Supplier A :
	 Material A : 20.0
	 Material B : 19.999999999999996
	 Material C : 0.0
	 Material D : 0.0
   Supplier A  Bill:  2800.0
   Supplier B :
	 Material A : 19.000000000000004
	 Material B : 4.000000000000001
	 Material C : 0.0
	 Material D : 0.0
   Supplier B  Bill:  2155.0000000000005
   Supplier C :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier C  Bill:  0.0
   Supplier D :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 14.000000000000002
	 Material D : 50.0
   Supplier D  Bill:  6440.0
   Supplier E :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier E  Bill:  0.0
Factory B :
   Supplier A :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier A  Bill:  0.0
   Supplier B :
	 Material A : 6.000000000000001
	 Material B : 34.0


In [17]:
# L Manufacturing Cost
print("Production Volume:")
print("****************************")

for factory in factories:
    print(factory,":")
    production_cost_total = 0
    for product in products:
        if production_volume[(factory, product)].solution_value() >0:
            print("  ",product,": ",production_volume[(factory, product)].solution_value())
            production_cost_total += production_volume[(factory, product)].solution_value() * production_cost.loc[product][factory]
    print("   Manufacturing cost: ", production_cost_total)


Production Volume:
****************************
Factory A :
   Product A :  6.0
   Product B :  1.000000000000001
   Product D :  3.0
   Manufacturing cost:  1010.0000000000001
Factory B :
   Product A :  2.0
   Product B :  0.999999999999999
   Product C :  4.0
   Manufacturing cost:  429.9999999999999
Factory C :
   Product A :  2.000000000000001
   Product D :  5.0
   Manufacturing cost:  425.0000000000001


In [18]:
# M Shipping Cost
print("\nShipping to Customer:") 
print("****************************")

for customer in customers:   
    shipping_cost = 0
    print(customer)
    for product in products:
        print("  ", product)
        for factory in factories: 
            print("\t",factory,": ",delivery[(factory, customer, product)].solution_value())
            shipping_cost += delivery[(factory, customer, product)].solution_value() * shipping_costs.loc[factory][customer]
    print("   Shipping Cost: ", shipping_cost)




Shipping to Customer:
****************************
Customer A
   Product A
	 Factory A :  4.999999999999999
	 Factory B :  0.0
	 Factory C :  2.000000000000001
   Product B
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product C
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product D
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  1.0
   Shipping Cost:  279.99999999999994
Customer B
   Product A
	 Factory A :  1.0
	 Factory B :  2.0
	 Factory C :  0.0
   Product B
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product C
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product D
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Shipping Cost:  110.0
Customer C
   Product A
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product B
	 Factory A :  1.000000000000001
	 Factory B :  0.9999999999999989
	 Factory C :  0.0
   Product C
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product D
	 Factory A 

In [19]:

#  Sub Task N   
print("\nMaterial Bifurcation and Cost per unit")
print("****************************")
#

for customer in customers:
    print(customer)
    for product in products:
        
        unit_cost_per_product = 0
        if int(customer_demand.loc[product][customer]) >0:
            print("  ", product)
            for factory in factories:
                
                if delivery[(factory, customer, product)].solution_value() >0:
                    print("\t", factory, ": ")
                    # Calculating the Shipping cost from factory to customer based on number of products
                    shipping_cost = delivery[(factory, customer, product)].solution_value() * shipping_costs.loc[factory][customer]
                    # Calculating the manufacturing cost 
                    man_cost = delivery[(factory, customer, product)].solution_value() * production_cost.loc[product][factory]
                    unit_cost_per_product += shipping_cost
                    unit_cost_per_product += man_cost
                    material_cost_to_customer = 0
                    for material in materials:
                        material_units = 0
                        material_units += delivery[(factory, customer, product)].solution_value() * production_req.loc[product][material]
                        
                        print("\t  ",material,": ", material_units)  
                        #raw material cost
                        material_cost = 0
                        #raw material cost
                        rshipping_cost = 0 
                        material_count = 0
                        for supplier in suppliers:
                            material_cost +=  orders[(factory, material, supplier)].solution_value() * raw_material_cost.loc[supplier][material]
                            rshipping_cost += orders[(factory, material, supplier)].solution_value() * raw_material_shipping.loc[supplier][factory]
                            material_count += orders[(factory, material, supplier)].solution_value()
                        material_cost_to_customer = ((material_cost + rshipping_cost)/material_count) * material_units
                        unit_cost_per_product += material_cost_to_customer
            print("\t cost per unit : ", unit_cost_per_product/int(customer_demand.loc[product][customer]))


Material Bifurcation and Cost per unit
****************************
Customer A
   Product A
	 Factory A : 
	   Material A :  24.999999999999996
	   Material B :  14.999999999999996
	   Material C :  0.0
	   Material D :  0.0
	 Factory C : 
	   Material A :  10.000000000000004
	   Material B :  6.000000000000003
	   Material C :  0.0
	   Material D :  0.0
	 cost per unit :  910.9752747252749
   Product D
	 Factory C : 
	   Material A :  3.0
	   Material B :  2.0
	   Material C :  4.0
	   Material D :  15.0
	 cost per unit :  3913.75
Customer B
   Product A
	 Factory A : 
	   Material A :  5.0
	   Material B :  3.0
	   Material C :  0.0
	   Material D :  0.0
	 Factory B : 
	   Material A :  10.0
	   Material B :  6.0
	   Material C :  0.0
	   Material D :  0.0
	 cost per unit :  745.7051282051283
Customer C
   Product B
	 Factory A : 
	   Material A :  0.0
	   Material B :  0.0
	   Material C :  2.000000000000002
	   Material D :  5.000000000000005
	 Factory B : 
	   Material A :  0.0
	